# Imports

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

# import webscrape package
from cryptory import Cryptory

# Twitter,Google,Blockchain Data from Bitinfocharts.com + Reddit Metrics + Yahoo Finance BTC, JPY Prices

In [2]:
# initialise object / set start date 
crypto = Cryptory(from_date="2017-12-31")

In [3]:
# daily btc tweet mentions
tweets = crypto.extract_bitinfocharts("btc", metric='tweets')
tweets.head()

,date,btc_tweets
0,2020-12-03,44375
1,2020-12-02,48309
2,2020-12-01,60008
3,2020-11-30,66042
4,2020-11-29,33292


In [4]:
# proxy for crypto adoption
activeaddresses = crypto.extract_bitinfocharts("btc", metric='activeaddresses')
activeaddresses.head()

,date,btc_activeaddresses
0,2020-12-03,995706
1,2020-12-02,870866
2,2020-12-01,1031232
3,2020-11-30,749325
4,2020-11-29,717058


In [5]:
# proxy for selling presssure in markets
mining_profitability = crypto.extract_bitinfocharts("btc", metric='mining_profitability')
mining_profitability.head()

,date,btc_mining_profitability
0,2020-12-03,0.152
1,2020-12-02,0.147
2,2020-12-01,0.160
3,2020-11-30,0.119
4,2020-11-29,0.128


In [6]:
transactionfees = crypto.extract_bitinfocharts("btc", metric='transactionfees')
transactionfees.head()

,date,btc_transactionfees
0,2020-12-03,8.106
1,2020-12-02,7.621
2,2020-12-01,7.412
3,2020-11-30,5.331
4,2020-11-29,2.723


In [7]:
mediantransactionfee = crypto.extract_bitinfocharts("btc", metric='median_transaction_fee')
mediantransactionfee.head()

,date,btc_median_transaction_fee
0,2020-12-03,4.545
1,2020-12-02,4.375
2,2020-12-01,4.259
3,2020-11-30,2.836
4,2020-11-29,1.195


In [8]:
mediantransactionvalue = crypto.extract_bitinfocharts("btc", metric='mediantransactionvalue')
mediantransactionvalue.head()

,date,btc_mediantransactionvalue
0,2020-12-03,622.343
1,2020-12-02,691.858
2,2020-12-01,845.304
3,2020-11-30,862.722
4,2020-11-29,476.560


In [9]:
# user growth of BTC subreddit
subs = crypto.extract_reddit_metrics("btc", "subscriber-growth")
subs.head()

,date,subscriber_growth
0,2020-12-02,244
1,2020-12-01,277
2,2020-11-30,310
3,2020-11-29,219
4,2020-11-28,255


In [10]:
# user growth of BTC subreddit
subgrowth = crypto.extract_reddit_metrics("btc", "subscriber-growth-perc")
subgrowth.head()

,date,subscriber_growth_perc
0,2020-12-02,0.000714
1,2020-12-01,0.000812
2,2020-11-30,0.000909
3,2020-11-29,0.000643
4,2020-11-28,0.000749


In [11]:
# google search trends
gtrends = crypto.get_google_trends(kw_list=['bitcoin','buy bitcoin','ETH','crypto'])
gtrends.head()

,date,bitcoin,buy bitcoin,ETH,crypto
0,2020-12-01,26.803553,0.322934,1.614672,2.906409
1,2020-11-30,26.803553,0.645869,1.937606,2.906409
2,2020-11-29,19.053128,0.322934,1.291738,2.583475
3,2020-11-28,20.344866,0.322934,1.291738,2.906409
4,2020-11-27,23.574210,0.645869,1.614672,2.906409


# Merge Alt dataframes

In [65]:
data_frames = [tweets,
               activeaddresses,
               mining_profitability,
               transactionfees,
               mediantransactionfee,
               mediantransactionvalue,
               subgrowth, 
               gtrends,  
               ]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['date'],
                                            how='outer'), data_frames).replace(to_replace=np.nan, method='bfill')

df_merged.rename(columns={'bitcoin': 'btc_gtrends',
                           'buy bitcoin': 'buybitcoin_gtrends',
                           'ETH': 'ETH_gtrends',
                           'crypto': 'crypto_gtrends',
                           'subscriber_growth_perc': 'sub_growth_%chg',
                           'date': 'Date'
                           },inplace=True)

df_merged.sort_index(ascending=False,inplace=True)
df_merged.set_index(df_merged['Date'], inplace=True)
df_merged.drop(columns='Date',inplace=True)
df_merged.head()

,btc_tweets,btc_activeaddresses,btc_mining_profitability,btc_transactionfees,btc_median_transaction_fee,btc_mediantransactionvalue,sub_growth_%chg,btc_gtrends,buybitcoin_gtrends,ETH_gtrends,crypto_gtrends
Date,,,,,,,,,,,
2017-12-31,64989,847972,2.281,25.170,15.291,2182.0,0.007039,48.214286,0.0,1.295976,6.291338
2018-01-01,62044,842391,2.174,26.274,14.625,2104.0,0.007324,46.428571,0.0,1.295976,6.291338
2018-01-02,77723,982159,2.431,26.522,12.574,3921.0,0.007731,53.571429,0.0,1.295976,10.485563
2018-01-03,79086,953442,2.737,28.461,15.529,4847.0,0.008254,55.357143,0.0,2.591952,10.485563
2018-01-04,74534,1071574,2.606,27.264,15.519,5033.0,0.008228,53.571429,0.0,2.591952,14.679788


In [66]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1070 entries, 2017-12-31 to 2020-12-03
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   btc_tweets                  1070 non-null   int64  
 1   btc_activeaddresses         1070 non-null   int64  
 2   btc_mining_profitability    1070 non-null   float64
 3   btc_transactionfees         1070 non-null   float64
 4   btc_median_transaction_fee  1070 non-null   float64
 5   btc_mediantransactionvalue  1070 non-null   float64
 6   sub_growth_%chg             1070 non-null   float64
 7   btc_gtrends                 1070 non-null   float64
 8   buybitcoin_gtrends          1070 non-null   float64
 9   ETH_gtrends                 1070 non-null   float64
 10  crypto_gtrends              1070 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 100.3 KB


In [68]:
# to csv for EDA
df_merged.to_csv('../thecap/Data/BTC_altdata.csv')

# Bring in BTC,JPY, Alt Data + Merge + Generate Final DF

In [95]:
# Bring in BTC price
path = './Data/BTC_daily_since_1-2016.csv'
dfbtc = pd.read_csv(path, index_col='Date', parse_dates=True)
dfbtc.head()

,Close,vol%_chg1d,vol%_chg3d,vol%_chg8d,range,range%,Target,Target_1d
Date,,,,,,,,
2018-01-08,15170.099609,0.160589,-0.227634,0.517258,1.163969,0.065188,0,0.0
2018-01-09,14595.400391,-0.095249,-0.090343,0.618859,1.074425,-0.076930,0,1.0
2018-01-10,14973.299805,0.110492,0.166066,0.098192,1.093644,0.017888,1,0.0
2018-01-11,13405.799805,-0.106304,-0.102086,-0.020021,1.145957,0.047834,0,1.0
2018-01-12,13980.599609,-0.270254,-0.275768,-0.446101,1.081456,-0.056286,1,1.0


In [109]:
# Bring in Jpy price
path = '../thecap/Data/Jpy_daily_since_12-2017.csv'
dffiat = pd.read_csv(path, index_col='Date', parse_dates=True)
dffiat.head()

,JPY%,Close_JPY
Date,,
2018-01-02,0.000923,112.769997
2018-01-03,-0.004664,112.244003
2018-01-04,0.003234,112.607002
2018-01-05,0.001563,112.782997
2018-01-08,0.003290,113.153999


In [110]:
# Bring in Alt data
path = '../thecap/Data/BTC_altdata.csv'
dfalt = pd.read_csv(path, index_col='Date', parse_dates=True)
dfalt.head()

,btc_tweets,btc_activeaddresses,btc_mining_profitability,btc_transactionfees,btc_median_transaction_fee,btc_mediantransactionvalue,sub_growth_%chg,btc_gtrends,buybitcoin_gtrends,ETH_gtrends,crypto_gtrends
Date,,,,,,,,,,,
2017-12-31,64989,847972,2.281,25.170,15.291,2182.0,0.007039,48.214286,0.0,1.295976,6.291338
2018-01-01,62044,842391,2.174,26.274,14.625,2104.0,0.007324,46.428571,0.0,1.295976,6.291338
2018-01-02,77723,982159,2.431,26.522,12.574,3921.0,0.007731,53.571429,0.0,1.295976,10.485563
2018-01-03,79086,953442,2.737,28.461,15.529,4847.0,0.008254,55.357143,0.0,2.591952,10.485563
2018-01-04,74534,1071574,2.606,27.264,15.519,5033.0,0.008228,53.571429,0.0,2.591952,14.679788


In [111]:
data_frames = [dfbtc,
               dffiat,
               dfalt  
               ]

df_final = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), data_frames).replace(to_replace=np.nan, method='bfill')
                                                                               
df_final.sort_index(ascending=True,inplace=True)

df_final.head()

,Close,vol%_chg1d,vol%_chg3d,vol%_chg8d,range,range%,Target,Target_1d,JPY%,Close_JPY,...,btc_activeaddresses,btc_mining_profitability,btc_transactionfees,btc_median_transaction_fee,btc_mediantransactionvalue,sub_growth_%chg,btc_gtrends,buybitcoin_gtrends,ETH_gtrends,crypto_gtrends
Date,,,,,,,,,,,,,,,,,,,,,
2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,847972,2.281,25.170,15.291,2182.0,0.007039,48.214286,0.0,1.295976,6.291338
2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,842391,2.174,26.274,14.625,2104.0,0.007324,46.428571,0.0,1.295976,6.291338
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000923,112.769997,...,982159,2.431,26.522,12.574,3921.0,0.007731,53.571429,0.0,1.295976,10.485563
2018-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004664,112.244003,...,953442,2.737,28.461,15.529,4847.0,0.008254,55.357143,0.0,2.591952,10.485563
2018-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003234,112.607002,...,1071574,2.606,27.264,15.519,5033.0,0.008228,53.571429,0.0,2.591952,14.679788


In [112]:
# Final df to csv for EDA
df_final.to_csv('../thecap/Data/final.csv')

In [113]:
df_final.tail()

,Close,vol%_chg1d,vol%_chg3d,vol%_chg8d,range,range%,Target,Target_1d,JPY%,Close_JPY,...,btc_activeaddresses,btc_mining_profitability,btc_transactionfees,btc_median_transaction_fee,btc_mediantransactionvalue,sub_growth_%chg,btc_gtrends,buybitcoin_gtrends,ETH_gtrends,crypto_gtrends
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-29,18177.484375,-0.045001,-0.492906,-0.214785,1.041261,-0.013752,1.0,1.0,-0.001535,104.082001,...,717058,0.128,2.723,1.195,476.560,0.000643,19.053128,0.322934,1.291738,2.583475
2020-11-30,19625.835938,0.533004,0.227379,0.156201,1.086418,0.043368,1.0,0.0,-0.001535,104.082001,...,749325,0.119,5.331,2.836,862.722,0.000909,26.803553,0.645869,1.937606,2.906409
2020-12-01,18802.998047,0.039917,0.522456,0.161263,1.083182,-0.002979,0.0,1.0,0.002085,104.299004,...,1031232,0.160,7.412,4.259,845.304,0.000812,26.803553,0.322934,1.614672,2.906409
2020-12-02,19201.091797,-0.246727,0.200866,-0.273596,1.052356,-0.028459,1.0,1.0,0.000096,104.308998,...,870866,0.147,7.621,4.375,691.858,0.000714,26.803553,0.322934,1.614672,2.906409
2020-12-03,19445.398438,-0.145967,-0.331001,-0.269502,1.033838,-0.017597,1.0,NaN,0.001428,104.458000,...,995706,0.152,8.106,4.545,622.343,0.000714,26.803553,0.322934,1.614672,2.906409
